In [2]:
import os

In [3]:
%pwd

'c:\\Users\\ojha.manish.kumar\\Documents\\Udemy\\DataScience_End_to_End_Projects\\data_science_mlops\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\ojha.manish.kumar\\Documents\\Udemy\\DataScience_End_to_End_Projects\\data_science_mlops'

In [16]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir:Path
    source_url:str
    local_data_file:Path
    unzip_dir:Path

In [17]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [35]:
class ConfigurationManager:
    def __init__(
        self, 
        config_file_path=CONFIG_FILE_PATH,
        params_file_path=PARAM_FILE_PATH,
        schema_file_path=SCHEMA_FILE_PATH
        ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion(self)->DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])    

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_url = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )            

        return data_ingestion_config


In [36]:
## DataIngestion Components
import os
from urllib import request
from src.datascience import logger
import zipfile

## Downloading the zip file
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download with following info:\n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """"
        zip_file_path:str
        Extracts the zip file into the data directory
        Function return None
        """

        unzip_path = self.config.unzip_dir
        os.makedir(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [40]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    #data_ingestion.extract_zip_file()
except Exception as e:
     raise e

[2025-09-16 11:36:40,805: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-09-16 11:36:40,805: INFO: common]: yaml file: params.yaml loaded successfully
[2025-09-16 11:36:40,805: INFO: common]: yaml file: schema.yaml loaded successfully
[2025-09-16 11:36:40,824: INFO: common]: created directory at: artifacts
[2025-09-16 11:36:40,827: INFO: common]: created directory at: artifacts/data_ingestion
[2025-09-16 11:36:41,715: INFO: 931786688]: artifacts/data_ingestion/data.csv download with following info:
Date: Tue, 16 Sep 2025 06:06:40 GMT
Server: Apache/ProXad [Jan 23 2019 20:05:46]
Last-Modified: Fri, 18 Mar 2016 09:38:13 GMT
ETag: "306028e-bc44-56ebcc85"
Connection: close
Accept-Ranges: bytes
Content-Length: 48196
Content-Type: text/comma-separated-values


